Check the GPU, make sure it is Tesla P100-PCIE or V100-SXM2

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Wed Jun 29 06:55:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Connect Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/markus-k/bisenetv2-tf2.git
!mv bisenetv2-tf2/* ./

Cloning into 'bisenetv2-tf2'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 104 (delta 53), reused 73 (delta 24), pack-reused 0
Receiving objects: 100% (104/104), 4.18 MiB | 6.91 MiB/s, done.
Resolving deltas: 100% (53/53), done.


##Import libraries
#### Reqirements
- keras >= 2.2.0 or tensorflow >= 1.13
- segmenation-models==1.0.*
- albumentations==0.3.0

In [ ]:
# %tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())

import datetime
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import matplotlib.pyplot as plt

import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optimizers

import tensorflow_datasets as tfds

from tensorflow.data import AUTOTUNE

from utils import display_dataset, predict_tf

import cv2
from PIL import Image
from skimage import io
import random as rng

# classification
from tensorflow.keras.models import Model
import scipy as sp
from google.colab.patches import cv2_imshow

2.8.2
/device:GPU:0


# Segmentation model

In [ ]:
from model import bisenetv2_compiled, bisenetv2_output_shape

INPUT_SHAPE = (256, 512, 3)
NUM_CLASSES = 2
SCALE = 8
OUTPUT_SHAPE = bisenetv2_output_shape(NUM_CLASSES, SCALE, input_shape=INPUT_SHAPE)
# create model
model = bisenetv2_compiled(num_classes=NUM_CLASSES, out_scale=SCALE, input_shape=INPUT_SHAPE)
# load weights
model.load_weights('drive/MyDrive/colab_data/chimei_2_views/checkpoints/BiSeNetV2/20220415_568_80/BiSeNetV2_568_1to80_11_0.92_weight.h5') 

tensorflow_addons not available, not using weight-decay


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 256, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 256, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

## h5 to TensorFlow SavedModel Format
https://ksingh7.medium.com/part-iv-convert-keras-model-to-tensorflow-savedmodel-format-205a168c0eef  
https://github.com/tensorflow/tensorflow/issues/43478  

In [ ]:
tf_savedmodel_folder = 'drive/MyDrive/colab_data/chimei_2_views/checkpoints/BiSeNetV2/20220415_568_80/BiSeNetV2_568_1to80_11_0.92_tfsavedmodel'
tf.keras.models.save_model(model, tf_savedmodel_folder)
print("successfully saved keras model file to tensorflow model file")

INFO:tensorflow:Assets written to: drive/MyDrive/colab_data/chimei_2_views/checkpoints/BiSeNetV2/20220415_568_80/BiSeNetV2_568_1to80_11_0.92_tfsavedmodel/assets
successfully saved keras model file to tensorflow model file


In [ ]:
model = tf.keras.models.load_model(tf_savedmodel_folder, compile=False)